<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
کمک به بحران برق
</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در این سوال از مسابقه با داده های جمع آوری شده از سنسورهای توربین های بادی سر و کار داریم. با توجه به توضیحات ذکر شده در صفحه سوال داده ها را دانلود کرده و به حل مسئله بپردازید:)
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
با استفاده از <a href="https://drive.google.com/file/d/162iKuz9w5vuFkG8vbXuTiFswthvL1qZT/view?usp=sharing">لینک</a> میتوانید به مجموعه داده های آموزش و آزمایش دسترسی داشته باشید و آنها را دانلود کنید.
</font> 
</p>


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import lightgbm as lgb

from scipy.stats import skew, kurtosis # Imported but unused in original, now incorporated for robustness

In [4]:

"""
AeroGen Dynamics - Predictive Maintenance
*** ENHANCED AND SCALABLE SOLUTION ***

Optimized script with:
1. Robust Null Value Handling (Missing Indicator Flags).
2. Advanced Feature Engineering (Statistical moments, Trend, Volatility).
3. Cost-Sensitive Learning (LightGBM with custom weights).
4. Time-Series Integrity (Mitigation of Label Leakage risks).
5. PySpark Placeholder (For true big data scalability, commented out).
"""

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import lightgbm as lgb
from scipy.stats import skew, kurtosis # Imported but unused in original, now incorporated for robustness

# --- Global Constants ---
RANDOM_STATE = 42
# The COST_MATRIX defines the business penalty/reward for classification
COST_MATRIX = np.array([
    [0, -2, -4, -8, -12],
    [-15, 20, -3, -6, -10],
    [-30, -15, 40, -5, -8],
    [-50, -30, -15, 80, -5],
    [-80, -50, -30, -15, 150]
])


# --- SCALABILITY NOTE ---
# If your dataset is truly 'huge' (10s to 100s of GBs), the Pandas operations below will
# crash due to memory or be too slow. You must use a distributed framework like PySpark.
# The code below is optimized for Pandas, but here is a placeholder for Spark:

# # from pyspark.sql import SparkSession
# # spark = SparkSession.builder.appName("AeroGen_Maintenance_Spark").getOrCreate()
# # def load_csv_spark(path):
# #     return spark.read.csv(path, header=True, inferSchema=True).cache()
# # If using Spark, the 'load_csv' and 'feature_engineering' functions below must be rewritten
# # using Spark DataFrames and Window functions.


def load_csv(path):
    """Loads CSV with memory optimization by downcasting numerical columns."""
    print(f"Loading {path} ...")
    # Read the data, ensuring 'object' dtype is handled
    df = pd.read_csv(path)

    # Optimize memory usage by downcasting
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        if 'time_step' not in col:
            # Use 'float32' for large datasets
            df[col] = pd.to_numeric(df[col], downcast='float')

    return df

# Load Data

# NOTE: Ensure these files exist in the execution environment
test_op =  pd.read_csv('test_operational_readouts.csv')
test_op

,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,...,397_26,397_27,397_28,397_29,397_30,397_31,397_32,397_33,397_34,397_35
0,1,4.4,94410.0,7982.0,3115020.0,2625.0,1304.0,366624.0,232040.0,147104.0,...,72548.0,9696.0,1309.0,7.0,399.0,31843.0,89138.0,24101.0,5112.0,7.0
1,1,5.0,105405.0,9178.0,3467240.0,3225.0,1305.0,390896.0,252528.0,157824.0,...,83147.0,12042.0,1841.0,7.0,455.0,35749.0,106366.0,28168.0,5420.0,7.0
2,1,8.2,214440.0,23348.0,7085790.0,8010.0,4897.0,461240.0,378264.0,313016.0,...,198090.0,26099.0,4516.0,21.0,1120.0,94017.0,293462.0,85596.0,16655.0,14.0
3,1,9.8,260190.0,29120.0,8696850.0,12525.0,6513.0,493728.0,455352.0,363984.0,...,244326.0,32708.0,5714.0,49.0,1498.0,118349.0,378176.0,115130.0,24054.0,35.0
4,1,20.6,519780.0,57967.0,17379560.0,32130.0,7617.0,665120.0,818912.0,664784.0,...,500995.0,61122.0,14247.0,169.0,3788.0,222712.0,741413.0,237274.0,59636.0,639.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198135,33638,59.6,1151970.0,53911.0,37965610.0,5760.0,2024.0,1060370.0,1621321.0,1296818.0,...,900593.0,59376.0,8906.0,70.0,11783.0,386958.0,717951.0,143395.0,17803.0,127.0
198136,33638,65.6,1321905.0,65091.0,43447361.0,6856.0,2024.0,1141978.0,1773113.0,1425914.0,...,1011299.0,69093.0,10390.0,126.0,13428.0,421160.0,812403.0,170877.0,21681.0,141.0
198137,33638,69.8,1438665.0,71591.0,47111501.0,7201.0,2024.0,1166667.0,1862506.0,1502034.0,...,1086320.0,73748.0,11454.0,133.0,14184.0,445289.0,880675.0,186809.0,24153.0,148.0
198138,33638,70.8,1459155.0,73008.0,47791981.0,7231.0,2024.0,1173795.0,1895626.0,1517987.0,...,1098011.0,74736.0,11672.0,133.0,14436.0,449272.0,890349.0,189582.0,24580.0,148.0


In [5]:
train_op =  pd.read_csv('train_operational_readouts.csv', on_bad_lines='skip' )


C:\Users\ASUS\AppData\Local\Temp\ipykernel_8464\1428763140.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,67,68,69,70,72,73,74,75,76,77,79,80,81,82,84,86,89,97,99,103) have mixed types. Specify dtype option on import or set low_memory=False.
  train_op =  pd.read_csv('train_operational_readouts.csv', on_bad_lines='skip' )


In [6]:
import pandas as pd
train_specs =  pd.read_csv('train_specifications.csv')
train_tte = pd.read_csv('train_tte.csv')




In [7]:
val_op =  pd.read_csv('validation_operational_readouts.csv')
val_specs =  pd.read_csv('validation_specifications.csv')

val_labels =  pd.read_csv('validation_labels.csv')

In [ ]:


# --- 1. Label Generation (Preventing Label Leakage / Temporal Bias) ---
# Your original function for mapping Time-To-Failure (TTF) to classes
def ttf_to_class(ttf):
    """Maps Time-To-Failure (hours) to a categorical maintenance class (0-4)."""
    if ttf > 48:
        return 0
    elif 24 < ttf <= 48:
        return 1
    elif 12 < ttf <= 24:
        return 2
    elif 6 < ttf <= 12:

        return 3
    else:
        return 4
    
# Merge TTE data onto the operational data
train_full = train_op.merge(train_tte, on='vehicle_id', how='left')

# Calculate the actual TTF for every time step *leading up* to the failure.
# This is a critical step to ensure every row has a label corresponding to its future state.
# For healthy (in_study_repair=0) or post-repair data, use a large placeholder.
train_full['time_to_failure'] = np.where(
    train_full['in_study_repair'] == 1,
    train_full['length_of_study_time_step'] - train_full['time_step'],
    999999 # Placeholder for healthy turbines (Class 0)
)
train_full['class_label'] = train_full['time_to_failure'].apply(ttf_to_class)

# --- 2. Feature Engineering (Enhanced Dimensionality Reduction and Stat Moments) ---

# Identify histogram columns (columns with the same numerical prefix)
def get_hist_cols(df):
    """Identifies columns that belong to aggregated histogram features."""
    hist_cols = {}
    for col in df.columns:
        if '_' in col:
            prefix = col.split('_')[0]
            # Assumes sensor columns are prefixed by numbers if they are histogram bins
            if prefix.isdigit():
                if prefix not in hist_cols:
                    hist_cols[prefix] = []
                hist_cols[prefix].append(col)
    return hist_cols

HISTOGRAM_COLUMNS = [c for v in get_hist_cols(train_full).values() for c in v]
NON_SENSOR_COLS = ['vehicle_id', 'time_step', 'length_of_study_time_step', 'in_study_repair', 'time_to_failure', 'class_label']
SENSOR_COLS = [c for c in train_full.columns if c not in NON_SENSOR_COLS and c not in HISTOGRAM_COLUMNS]

def feature_engineering(df):
    print("Starting Feature Engineering...")
    object_cols_to_clean = df.select_dtypes(include =['object']).columns
    for col_name in object_cols_to_clean :
        df [col_name] = df[col_name].astype(str).str.strip()
        df [col_name]=pd.to_numeric(df[col_name],errors='coerce')

    
    for prefix, cols in get_hist_cols(df).items():
        if not cols: continue

        # Mean/Spread/Total Activity (Original Features)
        df[f'hist_{prefix}_std'] = df[cols].std(axis=1)
        df[f'hist_{prefix}_sum'] = df[cols].sum(axis=1)
        # New: Min/Max to capture range of activity in the histogram
        df[f'hist_{prefix}_min'] = df[cols].min(axis=1)
        df[f'hist_{prefix}_max'] = df[cols].max(axis=1)


    # Drop individual bins after feature extraction to save memory (Crucial for large data)
    df.drop(columns=HISTOGRAM_COLUMNS, errors='ignore', inplace=True)
      # 2.2. Time Series Features (Lag, Rolling Statistics, and Moments)
    lag_roll_cols = SENSOR_COLS + [c for c in df.columns if c.startswith('hist_') and c.endswith(('_mean', '_sum'))]

    # Define rolling window size
    ROLLING_WINDOW = 10

    for col in lag_roll_cols:
        # Lag Features (Change since last time step - Captures instantaneous velocity/trend)
        df[f'{col}_lag1'] = df.groupby('vehicle_id')[col].diff(1)
        df[f'{col}_lag3'] = df.groupby('vehicle_id')[col].diff(3) # Deeper lag
        # Rolling Mean and STD (Trend and Stability over the last N steps)
        # Using min_periods=1 is important to get values even at the start of a sequence
        roll_mean = df.groupby('vehicle_id')[col].transform(lambda x: x.rolling(ROLLING_WINDOW, min_periods=1).mean())
        roll_std = df.groupby('vehicle_id')[col].transform(lambda x: x.rolling(ROLLING_WINDOW, min_periods=1).std())
        df[f'{col}_roll{ROLLING_WINDOW}_mean'] = roll_mean
        df[f'{col}_roll{ROLLING_WINDOW}_std'] = roll_std
        # New: Statistical Moments (Skewness and Kurtosis) - Captures shape of distribution over time
        # Skewness: Measures the asymmetry of the data distribution (critical for anomaly detection)
        # Kurtosis: Measures the "tailedness" of the distribution (critical for large/outlier events)
        df[f'{col}_roll{ROLLING_WINDOW}_skew'] = df.groupby('vehicle_id')[col].transform(lambda x: x.rolling(ROLLING_WINDOW, min_periods=ROLLING_WINDOW//2).apply(lambda y: skew(y, nan_policy='omit'), raw=True))
        df[f'{col}_roll{ROLLING_WINDOW}_kurt'] = df.groupby('vehicle_id')[col].transform(lambda x: x.rolling(ROLLING_WINDOW, min_periods=ROLLING_WINDOW//2).apply(lambda y: kurtosis(y, nan_policy='omit'), raw=True))
        # New: Mean/Std Ratio (A normalized measure of volatility)
        # Handle division by zero for stability
        df[f'{col}_volatility_ratio'] = np.where(roll_std != 0, roll_mean / roll_std, 0)

    return df
# Apply feature engineering to all datasets
train_full = feature_engineering(train_full.copy())
val_op = feature_engineering(val_op.copy())
test_op = feature_engineering(test_op.copy())

  

Starting Feature Engineering...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_8464\191300169.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'hist_{prefix}_max'] = df[cols].max(axis=1)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_8464\191300169.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'hist_{prefix}_std'] = df[cols].std(axis=1)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_8464\191300169.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

KeyboardInterrupt: 

In [10]:

# --- 3. Final Sample Selection (Latest time step only for Val/Test) ---

# Select last row for Validation and Test as required by the submission format
val_final = val_op.groupby('vehicle_id' ,group_keys=False).apply(lambda g: g.loc[g['time_step'].idxmax()]).reset_index(drop=True)
test_final = test_op.groupby('vehicle_id',group_keys=False).apply(lambda g: g.loc[g['time_step'].idxmax()]).reset_index(drop=True)

# Select ALL rows for Training to maximize samples for the model
train_final = train_full.copy()
# Remove initial NaN values created by rolling/lag features, which are usually a small percentage
train_final.dropna(subset=['class_label'], inplace=True)

# --- 4. Merge Specifications and Preprocessing (Robust Imputation) ---

# Merge specifications
train_final = train_final.merge(train_specs, on='vehicle_id', how='left')
val_final = val_final.merge(val_specs, on='vehicle_id', how='left')
# Use train_specs for test data as in original, assuming similar structural components
test_final = test_final.merge(train_specs, on='vehicle_id', how='left')

# Handle Categorical Columns (Label Encoding is fine for LightGBM)
categorical_cols = train_specs.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    le = LabelEncoder()
    # Combine all data for consistent encoding
    full_data = pd.concat([train_final[col], val_final[col], test_final[col]]).astype(str).fillna('missing')
    le.fit(full_data)

    for df in [train_final, val_final, test_final]:
        df[col] = df[col].astype(str).fillna('missing')
        df[col] = le.transform(df[col])

# Prepare Train/Validation Sets
EXCLUDE_COLS = NON_SENSOR_COLS + ['length_of_study_time_step', 'in_study_repair', 'time_to_failure']
X_train = train_final.drop(columns=EXCLUDE_COLS, errors='ignore')
y_train = train_final['class_label'].astype(int)

# Align validation labels
val_labels.set_index('vehicle_id', inplace=True)
val_final = val_final[val_final['vehicle_id'].isin(val_labels.index)]
y_val = val_labels.loc[val_final['vehicle_id']]['class_label'].astype(int)
X_val = val_final.drop(columns=['vehicle_id', 'time_step'], errors='ignore')

# Align test data
X_test = test_final.drop(columns=['vehicle_id', 'time_step'], errors='ignore')

# Impute and Scale Numerical Columns (Addressing the 'Null Value' concern)
num_cols = X_train.select_dtypes(include=[np.number]).columns

C:\Users\ASUS\AppData\Local\Temp\ipykernel_8464\3041010980.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_final = val_op.groupby('vehicle_id' ,group_keys=False).apply(lambda g: g.loc[g['time_step'].idxmax()]).reset_index(drop=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_8464\3041010980.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_final = test_op.groupby('vehicle_id',group_keys=False).apply(la

In [11]:

for c in num_cols:
    median_val = X_train[c].median()

    # 1. Add Missing Indicator Flags (Robust Null Handling)
    # This is critical to signal to the model that the null value was structurally missing,
    # which is often more informative than the imputed value itself.
    for df in [X_train, X_val, X_test]:
        # Check if column exists (some may be dropped) and has nulls
        if c in df.columns and df[c].isnull().any():
            df[f'{c}_is_missing'] = df[c].isnull().astype(int)

    # 2. Impute with Median (Original method)
    X_train[c] = X_train[c].fillna(median_val)
    X_val[c] = X_val[c].fillna(median_val)
    X_test[c] = X_test[c].fillna(median_val)

scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# --- 5. Cost-Sensitive LightGBM Model (Handling Imbalance) ---

# Calculate Class Weights based on inverse frequency (Baseline for Imbalance)
class_counts = y_train.value_counts().sort_index()
total_samples = len(y_train)
# Base weight calculation: total_samples / (num_classes * class_count)
base_weights = total_samples / (5 * class_counts)

# Manual Adjustment based on COST_MATRIX (High penalty for misclassifying 3 and 4)
# This step uses the base frequency weight and scales it further by the business risk.
custom_weights = np.array([
    base_weights.get(0, 1) * 1.0,   # Class 0 (Lowest Risk/Highest Frequency) - Low scale factor
    base_weights.get(1, 1) * 2.0,   # Class 1
    base_weights.get(2, 1) * 3.0,   # Class 2
    base_weights.get(3, 1) * 10.0,  # Class 3 (High Risk, severe penalty for missing) - High scale factor
    base_weights.get(4, 1) * 20.0   # Class 4 (Critical, catastrophic penalty for missing) - Highest scale factor
])

# Create sample weights array, mapped to each sample in the training data
sample_weights = y_train.apply(lambda x: custom_weights[x])
print(f"\nCustom Class Weights Applied: {custom_weights}")

# LightGBM Parameters - Optimized for Classification
params = {
    'objective': 'multiclass',
    'num_class': 5,
    'metric': 'multi_logloss',
    'learning_rate': 0.02, # Reduced learning rate for better convergence and precision
    'num_leaves': 40,      # Increased complexity
    'feature_fraction': 0.8, # Use subset of features on each tree
    'bagging_fraction': 0.8, # Use subset of data on each tree
    'bagging_freq': 1,
    'max_depth': 8,        # Deeper trees
    'seed': RANDOM_STATE,
    'n_jobs': -1,
    'verbosity': -1,
    'boosting_type': 'gbdt'
}

train_data = lgb.Dataset(X_train, label=y_train, weight=sample_weights)
val_data = lgb.Dataset(X_val, label=y_val)

print("\n--- Starting LightGBM Training (Cost-Sensitive) ---")
model = lgb.train(params,
                  train_data,
                  valid_sets=[val_data],
                  num_boost_round=3000, # Increased rounds
                  callbacks=[lgb.early_stopping(stopping_rounds=150, verbose=True),
                             lgb.log_evaluation(period=100)])


Custom Class Weights Applied: [2.04959873e-01 3.40102564e+01 1.03355844e+02 6.57950396e+02
 1.11322857e+03]

--- Starting LightGBM Training (Cost-Sensitive) ---


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: 171_0: object, 666_0: object, 427_0: object, 837_0: object, 167_0: object, 167_1: object, 167_2: object, 167_3: object, 167_4: object, 167_5: object, 167_6: object, 167_7: object, 167_8: object, 167_9: object, 309_0: object, 272_0: object, 272_1: object, 272_2: object, 272_3: object, 272_4: object, 272_5: object, 272_6: object, 272_7: object, 272_8: object, 272_9: object, 835_0: object, 370_0: object, 291_0: object, 291_1: object, 291_2: object, 291_3: object, 291_4: object, 291_5: object, 291_6: object, 291_7: object, 291_8: object, 291_9: object, 291_10: object, 158_0: object, 158_1: object, 158_2: object, 158_3: object, 158_4: object, 158_5: object, 158_6: object, 158_7: object, 158_8: object, 158_9: object, 100_0: object, 459_0: object, 459_1: object, 459_2: object, 459_3: object, 459_4: object, 459_5: object, 459_6: object, 459_7: object, 459_8: object, 459_9: object, 459_10: object, 459_11: objec

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    پیش‌بینی مدل خود بر روی داده‌های آزمایش (<code>test</code>) را در قالب یک <code>dataframe</code> در متغیری با نام <code>submission</code>ذخیره کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|class_label|vehicle_id|
|---|---|
|?|1 |
    
</font>


In [ ]:
# --- 6. Evaluation and Submission ---

y_pred = np.argmax(model.predict(X_val), axis=1)

def compute_raw_score(actual, predicted, cost_matrix):
    """Calculates the final cost-sensitive score based on the business matrix."""
    score = 0
    # Ensure arrays are correctly typed for indexing
    actual = np.array(actual, dtype=int)
    predicted = np.array(predicted, dtype=int)

    # Use advanced indexing for efficient calculation (vectorized operation)
    score = np.sum(cost_matrix[actual, predicted])
    return score

raw = compute_raw_score(y_val, y_pred, COST_MATRIX)
perfect_raw = compute_raw_score(y_val, y_val, COST_MATRIX)

print("\n--- Validation Results (Cost-Sensitive Score) ---")
# Requires sklearn.metrics.classification_report
try:
    print(classification_report(y_val, y_pred))
except NameError:
    # Handle case where classification_report might not be imported
    pass
print('Validation Confusion Matrix:\n', confusion_matrix(y_val, y_pred))
print('Validation Raw Score:', raw)
print('Validation Perfect Raw Score (Benchmark):', perfect_raw)
print('Validation Final Score (%):', 100 * max(0, raw) / perfect_raw if perfect_raw > 0 else 0)

# Final Prediction and Submission
test_pred = np.argmax(model.predict(X_test), axis=1)
sub = pd.DataFrame({'vehicle_id': test_final['vehicle_id'], 'class_label': test_pred.astype(int)})
sub.sort_values('vehicle_id', inplace=True)
sub.to_csv('submission.csv', index=False)
print('\nSaved submission.csv with final predictions.')


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [ ]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'notebook.ipynb')):
    %notebook -e notebook.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['notebook.ipynb', 'submission.csv']
compress(file_names)